In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
import re
import sys

import os
print(os.getcwd())   # Prints the current working directory

In [ ]:
# Provide the new path here
os.chdir('C:\\Users\\user\\Desktop\\Twitter NLP Classification Model\\DATASET\\dataset 31-05-2020')
df = pd.read_csv('result_prepro3.csv')

In [ ]:
print(df.head())

In [ ]:
print(len(df))

In [ ]:
####
# RANDOM SAMPLE
###
chosen_idx = np.random.choice(len(df), replace=False, size=10000)
df_sample = df.iloc[chosen_idx]

In [ ]:
# df_unigram = df.copy() ## USING ORIGINAL DATA 
df_unigram = df_sample.copy() ## USING SAMPLE DATA
df_unigram = df_unigram.assign(Text6 = df['Text5'])
df_unigram = df_unigram.assign(Binary1 = 0 )
df_unigram = df_unigram.assign(Binary2 = 0 )
df_unigram = df_unigram.assign(tfidf = 0)
df_unigram['Binary1'] = df_unigram['Class'].apply(lambda x: 0 if 'hoax' in x else 1)
df_unigram['Binary2'] = df_unigram['Class'].apply(lambda x: 1 if 'hoax' in x else 0)
df_unigram = df_unigram[['Text6','Binary1','Binary2','tfidf']]

df_bigram = df_unigram.copy()
df_trigram = df_unigram.copy()


In [ ]:
############################################################################
# IMPORTANT , PLEASE SET OUTPUT to be either 1 or 2 columns deliberately
############################################################################

##Output combine binary 1 + binary 2 [SELECT 1 COLUMN of OUTPUT]
# output = df_unigram[['Binary1','Binary2']].to_numpy()

## Output is Binary2 [SELECT 2 COLUMNs of OUTPUT]
output = df_unigram[['Binary2']].to_numpy()

In [ ]:
print(output.shape)

In [ ]:
from collections import Counter
from nltk.util import ngrams

In [ ]:
data = df_unigram['Text6'].copy()
print(data.head())
data_unigram = data.apply(lambda row: nltk.word_tokenize(str(row)))
print(data_unigram[0:10])

In [ ]:
import io
import time
from datetime import timedelta

from nltk.tokenize import sent_tokenize, word_tokenize


#You can suppress the message with this code before importing gensim:

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim

import time
import multiprocessing
from datetime import timedelta

from gensim.models import word2vec


from gensim.test.utils import common_texts
from gensim.models import Phrases



In [ ]:
######################################################################
# SET SMALL (for trial and error purpose) OR LARGE DATA
######################################################################

small_data_status = 1

if small_data_status == 1 :
    data_input = data_unigram[0:40]
elif small_data_status == 0 :
    data_input = data_unigram

In [ ]:
embedded_matrix_size = 3

#Word2Vec
# Create CBOW model 
model_unigram = gensim.models.Word2Vec(data_input, min_count=1, size = embedded_matrix_size, workers=multiprocessing.cpu_count()-1, window = 3 , sg =0)



In [ ]:
# print(model.syn1)
# print(model.syn2)
print(model_unigram.syn1neg) #negative sampling, embedding output matrix word2vec
print(type(model_unigram.syn1neg))
print(model_unigram.syn1neg.shape)
print(model_unigram.syn1neg.shape[0])



In [ ]:
bigram_transformer = Phrases(list(data_input))
model_bigram = gensim.models.Word2Vec(bigram_transformer[list(data_input)], min_count=1,size = 3)
print(model_bigram.syn1neg)

data_bigram = data_input.apply(lambda row:bigram_transformer[row])
print(type(data_bigram))
print(data_bigram)
data_word2vec_bigram = data_bigram[0:2].apply(lambda x: model_bigram[x])
print(data_word2vec_bigram)

In [ ]:
bigram_sentences = data_input.apply(lambda row: u' '.join(bigram_transformer[row]))    
trigram_transformer = Phrases(list(bigram_sentences))
model_trigram = gensim.models.Word2Vec(trigram_transformer[list(bigram_sentences)], min_count=1,size = 3)
print(model_trigram.syn1neg)



data_trigram = bigram_sentences.apply(lambda row:trigram_transformer[row])
print(type(data_trigram))
print(data_trigram)
data_word2vec_trigram = data_trigram[0:2].apply(lambda x: model_trigram[x])
print(data_word2vec_trigram)


In [ ]:
data_word2vec_unigram = data_input.apply(lambda x: model_unigram[x])
data_word2vec_bigram = data_bigram.apply(lambda x: model_bigram[x])
data_word2vec_trigram = data_trigram.apply(lambda x: model_trigram[x])

print(data_word2vec_unigram)
data_word2vec_unigram2 = np.array(data_word2vec_unigram)
print(data_word2vec_unigram2)


print(data_word2vec_bigram)
data_word2vec_bigram2 = np.array(data_word2vec_bigram)
print(data_word2vec_unigram2)

print(data_word2vec_trigram)
data_word2vec_trigram2 = np.array(data_word2vec_trigram)
print(data_word2vec_trigram2)


In [ ]:
## Sum all vectors

data_word2vec_unigram3 = np.zeros(data_word2vec_unigram2.shape)
data_word2vec_unigram3 = list(data_word2vec_unigram3)
print(type(data_word2vec_unigram3))

i = 0
for x in data_word2vec_unigram2 :
    data_word2vec_unigram3[i] = np.add.reduce(x)
#     print(x)
    i = i + 1

    
######################################## Bigram    
data_word2vec_bigram3 = np.zeros(data_word2vec_bigram2.shape)
data_word2vec_bigram3 = list(data_word2vec_bigram3)
print(type(data_word2vec_bigram3))

i = 0
for x in data_word2vec_bigram2 :
    data_word2vec_bigram3[i] = np.add.reduce(x)
#     print(x)
    i = i + 1 
    
################################################## Trigram    
data_word2vec_trigram3 = np.zeros(data_word2vec_trigram2.shape)
data_word2vec_trigram3 = list(data_word2vec_trigram3)
print(type(data_word2vec_trigram3))

i = 0
for x in data_word2vec_trigram2 :
    data_word2vec_trigram3[i] = np.add.reduce(x)
#     print(x)
    i = i + 1

In [ ]:
print(data_word2vec_unigram3)
print(data_word2vec_unigram3[0])
print(len(data_word2vec_unigram3))
data_word2vec_unigram4 = np.array(data_word2vec_unigram3)
print(data_word2vec_unigram4)


############### Bigram
print(data_word2vec_bigram3)
print(data_word2vec_bigram3[0])
print(len(data_word2vec_bigram3))
data_word2vec_bigram4 = np.array(data_word2vec_bigram3)
print(data_word2vec_bigram4)

################ Trigram
print(data_word2vec_trigram3)
print(data_word2vec_trigram3[0])
print(len(data_word2vec_trigram3))
data_word2vec_trigram4 = np.array(data_word2vec_trigram3)
print(data_word2vec_trigram4)

In [ ]:
print(data_word2vec_unigram4.shape)

In [ ]:
# #Variable Input , output and number of neurons for ANN

row_train_end = round(data_word2vec_unigram4.shape[0] * 0.8)
row_test_end = data_word2vec_unigram4.shape[0]

input_word2vec_unigram_training = data_word2vec_unigram4[0:row_train_end]
input_word2vec_unigram_testing = data_word2vec_unigram4[row_train_end:row_test_end]
output_word2vec_unigram_training = output[0:row_train_end]            #80%
output_word2vec_unigram_testing = output[row_train_end:row_test_end]  #20%
print(input_word2vec_unigram_training.shape)

##################################### Bigram
input_word2vec_bigram_training = data_word2vec_bigram4[0:row_train_end]
input_word2vec_bigram_testing = data_word2vec_bigram4[row_train_end:row_test_end]
output_word2vec_bigram_training = output[0:row_train_end]               #80%
output_word2vec_bigram_testing = output[row_train_end:row_test_end]     #20%

##################################### Trigram
input_word2vec_trigram_training = data_word2vec_trigram4[0:row_train_end]
input_word2vec_trigram_testing = data_word2vec_trigram4[row_train_end:row_test_end]
output_word2vec_trigram_training = output[0:row_train_end]                #80%
output_word2vec_trigram_testing = output[row_train_end:row_test_end]      #20%



In [ ]:
###############################################################################
# SET UNI-GRAM, BI-GRAM, TRI-GRAM IN THIS PART
###############################################################################

# n_grams = 1 # UNI-GRAM
n_grams = 2 # BI-GRAM
# n_grams = 3 # TRI-GRAM

if n_grams == 1:
    input_training_ANN = input_word2vec_unigram_training 
    input_testing_ANN = input_word2vec_unigram_testing
    output_training_ANN = output_word2vec_unigram_training
    output_testing_ANN = output_word2vec_unigram_testing
    print("Unigram Selected ")
    
elif n_grams == 2:
    input_training_ANN = input_word2vec_bigram_training
    input_testing_ANN = input_word2vec_bigram_testing
    output_training_ANN = output_word2vec_bigram_training
    output_testing_ANN = output_word2vec_bigram_testing
    print("Bigram Selected ")
        
elif n_grams == 3:
    input_training_ANN = input_word2vec_trigram_training
    input_testing_ANN = input_word2vec_trigram_testing
    output_training_ANN = output_word2vec_trigram_training
    output_testing_ANN = output_word2vec_trigram_testing
    print("Trigram Selected ")

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import seaborn as sns

import tensorflow as tf 
#import tensorflow_text as text 


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from tensorflow.keras import initializers
from tensorflow.python.ops.init_ops import glorot_normal_initializer
from tensorflow.python.ops.init_ops import glorot_uniform_initializer
from tensorflow.python.ops.init_ops import he_normal
from tensorflow.python.ops.init_ops import he_uniform
from tensorflow.python.ops.init_ops import truncated_normal_initializer

import numpy as np
import matplotlib.pyplot as plt
import array 
import pandas as pd

from scipy.sparse import coo_matrix

In [ ]:
# tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=123)
# BEGIN CODE TENSORFLOW 
seed_id_number = 123
tf.random.set_seed(seed_id_number)
def build_model():
    model = keras.Sequential([    
    layers.Dense(500, activation='relu' ,
                 input_shape=[embedded_matrix_size],
#                  kernel_initializer = truncated_normal_initializer(mean=0.0, stddev=0.05, seed=seed_id_number),
                 kernel_initializer = he_normal(seed=seed_id_number),
                 bias_initializer = he_normal(seed=seed_id_number)
                ), #1st hidden layer
    layers.Dense(350, activation='relu',
                 kernel_initializer = he_normal(seed=seed_id_number),
                 bias_initializer = he_normal(seed=seed_id_number)
                ), #2nd hidden layer
    layers.Dense(200, activation='relu',
                 kernel_initializer = he_normal(seed=seed_id_number),
                 bias_initializer = he_normal(seed=seed_id_number)
                ), #3rd hidden layer
    layers.Dense(1, activation ='sigmoid',
                kernel_initializer = glorot_normal_initializer(seed=seed_id_number),
                bias_initializer = glorot_normal_initializer(seed=seed_id_number)
                ), #output layer
    layers.Dropout(0.5)
    ])

  # (Learning Rate, Momentum,nesterov)
    optimizer = tf.keras.optimizers.SGD(0.01,0.9,False) 

    model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error','accuracy'])
    return model 

In [ ]:
#Create ANN Model
model = build_model()
model.summary()

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')
##Set training Iterations
# EPOCHS = 10
EPOCHS = 200
#Run Training Model
history = model.fit(
    input_training_ANN,output_training_ANN,
    epochs=EPOCHS, validation_split = 0.2, verbose=1,
    callbacks=[PrintDot()]
)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail() 

In [ ]:
 ##Print Weight and Bias for each layers
weight_1, bias_1 = model.layers[0].get_weights()
weight_2, bias_2 = model.layers[1].get_weights() 
weight_3, bias_3 = model.layers[2].get_weights()
weight_4, bias_4 = model.layers[3].get_weights()
# weight_5, bias_5 = model.layers[4].get_weights()

print('1st layer weight :\n',weight_1)
print('1st layer bias : \n',bias_1)

print('2nd layer weight : \n',weight_2)
print('2nd layer bias : \n',bias_2)

print('3rd layer weight : \n',weight_3)
print('3rd layer bias : \n',bias_3)

print('4rd layer weight : \n',weight_4)
print('4rd layer bias : \n',bias_4)

# print('4rd layer weight : \n',weight_5)
# print('4rd layer bias : \n',bias_5)

In [ ]:
#Set Plot 'Module'
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    
    ## Plot Target Output Graph
    fig = plt.figure()
    fig.suptitle(' [TARGET OUTPUT TESTING]')
    plt.plot(output_testing_ANN)
    
    ##Plot Mean Absolute Error vs Epoch Graph (test and train)
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Validation Error')
    plt.legend()
    
    
    ##Plot Mean Square Error vs Epoch graph (test and train)
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['loss'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Validation Error')
    plt.legend()




  
  
##Calling plot 'Module'   
plot_history(history)
# plot_history(hist)



In [ ]:
#Run Testing Model
test_predictions = model.predict(input_testing_ANN)

print(len(test_predictions))
test_predictions1 = test_predictions[0]
print(test_predictions1)

test_predictions1[test_predictions1 >= 0.5] = 1
test_predictions1[test_predictions1 < 0.5] = 0

print(output_testing_ANN)
print(test_predictions)




In [ ]:
plt.figure()
plt.suptitle('Output Prediction')
plt.plot(test_predictions)

print(('Test Predictions = {}').format(test_predictions))

test_predictions[test_predictions >= 0.5] = 1
test_predictions[test_predictions < 0.5] = 0
print(test_predictions)

##Plot Prediction Error [MPG] graph
error = test_predictions - output_testing_ANN

plt.figure()
plt.hist(error, bins = 25, label = "Binary1")
#plt.hist(error[:,1], bins = 25, label = "Binary2" )
plt.xlabel("Prediction Error [MPG]")
plt.ylabel("Count")
plt.legend(loc="upper right")
plt.show()

In [ ]:
print(test_predictions[0])
print(test_predictions[:,0])
print(type(test_predictions[:,0]))

In [ ]:
print(type(test_predictions1))
print(type(df[40516:50644]))


# if small_data_status == 1:
#     df_testing = df.iloc[11:20,:].copy()

# elif small_data_status == 0:
#     df_testing = df.iloc[40516:50644,:].copy()
df_testing = df.iloc[row_train_end:row_test_end,:].copy()

In [ ]:
print(df_testing.head())

In [ ]:
df_testing2 =  pd.DataFrame(test_predictions[:,0] , columns =['Test Predictions Binary1'])
df_testing2['Test Predictions Binary2'] =  pd.DataFrame(test_predictions[:,0])
df_testing2['Text5'] = df_testing['Text5'].to_numpy()
print(df_testing2.head())

In [ ]:
# df_testing3 = df_testing2[df_testing2['Test Predictions Binary2']==0] # Filter Non-Hoax Prediction
df_testing3 = df_testing2[df_testing2['Test Predictions Binary2']==1] # Filter Hoax Prediction
print(df_testing3.head())
print(df_testing3.size)

In [ ]:
words = list(df_testing3['Text5'])
words = np.array(words)
print(type(words))
print(words)

In [ ]:
#confusion Matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
matrix = confusion_matrix(test_predictions.argmax(axis=1),output_testing_ANN.argmax(axis=1))
print(matrix)


In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_unigram = TfidfVectorizer(ngram_range=(1,1)) 
tfidf_bigram = TfidfVectorizer(ngram_range=(2,2))
tfidf_trigram = TfidfVectorizer(ngram_range=(3,3))
# input_word2vec_trigram_testing_small

In [ ]:
vectors_unigram = tfidf_unigram.fit_transform(df_unigram['Text6'].values.astype('U'))
vectors_bigram = tfidf_bigram.fit_transform(df_bigram['Text6'].values.astype('U'))
vectors_trigram = tfidf_trigram.fit_transform(df_trigram['Text6'].values.astype('U'))

In [ ]:
print(vectors_unigram.shape)

In [ ]:
print(output_word2vec_unigram_testing.shape)

In [ ]:
# vectors_unigram_testing_small = vectors_unigram[11:20]
# vectors_bigram_testing_small = vectors_bigram[11:20]
# vectors_trigram_testing_small = vectors_trigram[11:20]

In [ ]:
vectors_unigram_testing = vectors_unigram[row_train_end:row_test_end]
vectors_bigram_testing = vectors_bigram[row_train_end:row_test_end]
vectors_trigram_testing = vectors_trigram[row_train_end:row_test_end]

In [ ]:
if n_grams == 1:
    vectors_testing_ANN = vectors_unigram[row_train_end:row_test_end]
    tfidf_ANN = tfidf_unigram
elif n_grams == 2:
    vectors_testing_ANN = vectors_bigram[row_train_end:row_test_end]
    tfidf_ANN = tfidf_bigram
elif n_grams == 3:
    vectors_testing_ANN = vectors_trigram[row_train_end:row_test_end]
    tfidf_ANN = tfidf_trigram

In [ ]:
chi2score = chi2(vectors_testing_ANN, output_testing_ANN)[0]

In [ ]:
print(chi2score.shape)

In [ ]:
wscores = list(zip(tfidf_ANN.get_feature_names(), chi2score))
wchi2 = sorted(wscores, key=lambda x:x[1])
print(wchi2[0:2])
wchi2 = [row for row in wchi2 if not np.isnan(row[1])]
print(wchi2[0:2])
    

In [ ]:
plt.figure(figsize=(15,10))
wscores = list(zip(tfidf_ANN.get_feature_names(), chi2score)) # List
wscores1 = [row for row in wscores if not np.isnan(row[1])]  # remove NAN value
# wchi2 = sorted(wscores, key=lambda x:x[1]) #with NAN values
wchi2 = sorted(wscores1, key=lambda x:x[1]) #without NAN values                    
topchi2 = list(zip(*wchi2[-50:]))
x = range(len(topchi2[1]))
labels = topchi2[0]
plt.barh(x,topchi2[1], align='center', alpha=0.2)
plt.plot(topchi2[1], x, '-o', markersize=5, alpha=0.8)
plt.yticks(x, labels)
plt.xlabel('$\chi^2$')

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_cm(labels, predictions, p = 0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

In [ ]:
print(test_predictions)
plot_cm(output_testing_ANN[:,0],test_predictions[:,0])
